In [0]:
import dlt
from pyspark.sql.functions import *

In [0]:
from cryptography.fernet import Fernet
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

In [0]:
key = dbutils.secrets.get(scope="PII-secret", key="pii-key")
fernet = Fernet(key)

Approch 1

In [0]:
@dlt.view(
    name = "product_combined_view"
    
)
def product_changes():
    product = dlt.readStream("training.piyush.product_silver").alias("product")        
    prodmodel = dlt.read("training.piyush.productmodel_silver")      
    category = dlt.read("training.piyush.productcategory_silver")              
    subcategory = dlt.read("training.piyush.productsubcategory_silver")
    prodmodelproddesccult = dlt.read('training.piyush.productmodelproductdescriptionculture_silver')
    productdesc = dlt.read('training.piyush.productdescription_silver')          

    product = product.withColumn("Product_Surrogate", sha2(concat_ws("||", col("product.ProductID"), col("product.ModifiedDate")), 512))

    product_mergeddf = product \
          .join(prodmodel, product["ProductModelID"] == prodmodel["ProductModelID"], "left")\
          .join(prodmodelproddesccult, prodmodelproddesccult["ProductModelID"] == prodmodel["ProductModelID"], "left")\
          .join(subcategory, product["ProductSubcategoryID"] == subcategory["ProductSubcategoryID"], "left")\
          .join(category, subcategory["ProductCategoryID"] == category["ProductCategoryID"], "left")\
          .join(productdesc, productdesc['ProductDescriptionID'] == prodmodelproddesccult['ProductDescriptionID'], "left")

    # product_mergeddf = product_mergeddf.withColumn("Product_Surrogate", sha2(concat_ws("||", col("product.ProductID"), col("product.ModifiedDate")), 512))  


    product_final = product_mergeddf.select(
          product_mergeddf['Product_Surrogate'],
          product["ProductID"].alias('ProductID'),
          product["Name"].alias("Product_Name"),
          product["ProductNumber"],
          product['MakeFlag'],
          product["FinishedGoodsFlag"],
          product["Color"],
          product['StandardCost'],
          product['SellStartDate'],
          product['SellEndDate'],
          product['ModifiedDate'].alias('Product_ModifiedDate'),
          product['Ingested_time'],
          prodmodel['ProductModelID'].alias('ProductModelID'),
          prodmodel['Name'].alias('ProductModel_Name'),
          subcategory["ProductSubcategoryID"].alias("ProductSubcategoryID"),
          subcategory["Name"].alias("Subcategory_Name"),
          category['ProductCategoryID'].alias("ProductCategoryID"),
          category["Name"].alias("ProductCategory_Name"),
          prodmodelproddesccult['CultureID'],
          productdesc['ProductDescriptionID'],
          productdesc['Description']
        
        #   lit(None).alias('is_active'),
          # current_timestamp().alias("event_timestamp"),
        #   lit(None).cast('timestamp').alias('effective_to')
).filter(col('ProductID').isNotNull())
    
    
    return product_final
  
dlt.create_streaming_table(
  name = 'product_dimension_gold'
)

dlt.apply_changes(
  name = "apply_the_changes_to_the_dim_gold_table",
  target = 'product_dimension_gold',
  source = 'product_combined_view',
  keys = ['ProductID'],
  sequence_by = 'Product_ModifiedDate',
  stored_as_scd_type = 2
)

  

In [0]:
def decrypt_val(val):
    if val is None:
        return None
    else:
        return fernet.decrypt(val.encode()).decode()
    
decrypt_udf = udf(decrypt_val, StringType())



@dlt.view(
       
    name = "customer_combined_view"
)
def customer_changes():
    customer = dlt.read("training.piyush.customer_silver").alias('customer') 
    customer = customer.withColumn('AccountNumber', decrypt_udf(col('AccountNumber')))       
    person = dlt.readStream("training.piyush.person_silver").alias('person')
    person = person.withColumn('PersonType', decrypt_udf(person['PersonType']))
    businessentityaddress = dlt.read("training.piyush.businessentityaddress_silver")  
    address = dlt.read("training.piyush.address_silver")  
    addresstype = dlt.read("training.piyush.addresstype_silver")  
    emailaddress = dlt.read("training.piyush.emailaddress_silver").alias('emailaddress')
    emailaddress = emailaddress.withColumn('EmailAddress', decrypt_udf(col('EmailAddress')))  
               

    customer = customer.withColumn("Customer_Surrogate", sha2(concat_ws("||", col('customer.CustomerID'), col('customer.ModifiedDate')), 256))
    
    customer_mergeddf = person \
       .join(customer, person["BusinessEntityID"] == customer["PersonID"], "left")\
       .join(businessentityaddress, person['BusinessEntityID'] == businessentityaddress['BusinessEntityID'], "left") \
       .join(addresstype, businessentityaddress['AddressTypeID'] == addresstype['AddressTypeID'], "left") \
       .join(address, businessentityaddress['AddressID'] == address['AddressID'], "left")\
       .join(emailaddress, person['BusinessEntityID'] == emailaddress['BusinessEntityID'], "left")

  

    customer_final = customer_mergeddf.select(
          customer["Customer_Surrogate"],
          customer["CustomerID"],
          customer["PersonID"].alias('Customer_PersonID'),
          customer["StoreID"],
          customer['TerritoryID'],
          customer['AccountNumber'],
          customer['ModifiedDate'],
          businessentityaddress['BusinessEntityID'],
          businessentityaddress['AddressTypeID'],
          businessentityaddress['AddressID'],
          person['BusinessEntityID'].alias('PersonID'),
          person['PersonType'],
          person['NameStyle'],
          person['FirstName'],
          person['MiddleName'],
          person['LastName'],
          person['ModifiedDate'].alias('Person_ModifiedDate'),
          # addresstype['AddressTypeID'],
          # address['AddressID'],
          address['AddressLine1'],
          address['AddressLine2'],
      #     address['StateProvinceID'],
      #     address['PostalCode'],
          emailaddress['EmailAddressID'],
          emailaddress['EmailAddress'],
          customer['rowguid'],
          customer['Ingested_time'],
          current_timestamp().alias("event_timestamp")
    ).filter(col('CustomerID').isNotNull())
        
    return customer_final

dlt.create_streaming_table(
  name = 'customer_dimension_gold',
  schema = """
    CustomerID int,
    Customer_PersonID int,
    StoreID double,
    TerritoryID int,
    AccountNumber string,   
    ModifiedDate timestamp,
    BusinessEntityID int,
    AddressTypeID int,
    AddressID int,
    PersonID int,
    PersonType string,
    NameStyle boolean,
    FirstName string,
    MiddleName string,
    LastName string,
    Person_ModifiedDate timestamp,
    AddressLine1 string,
    AddressLine2 string,
    EmailAddressID int,
    EmailAddress string Mask training.piyush.mask_email_data,
    rowguid string,
    Ingested_time timestamp,
    event_timestamp timestamp,
    Customer_Surrogate string,
    __START_AT date,
    __END_AT date   
  """
)

dlt.apply_changes(
  name = "apply_the_changes_to_the_dim_customer_gold_table",
  target = 'customer_dimension_gold',
  source = 'customer_combined_view',
  keys = ['PersonID'],
  sequence_by = 'Person_ModifiedDate',
  stored_as_scd_type = 2
)

In [0]:
from pyspark.sql.functions import col, year, month, dayofmonth

@dlt.table(

    name = "date_dimension"

)
def salesorder_date_dimension():
    salesorderdetail = dlt.readStream("training.piyush.salesorderdetail_silver") 
    salesorderdetaildf = salesorderdetail.select(col("ModifiedDate").alias("date"))     
    
    return salesorderdetaildf.distinct().select(
                                    col("date").alias("ModifiedDate"),
                                    year("date").alias("year"),
                                    month("date").alias("month"),
                                    dayofmonth("date").alias("day")
    )



In [0]:
@dlt.table(
     name='Sales_fact_table',
    #  comment="Fact table for sales with FK to product, customer, and date dimensions"
 )
def fact_sales():
     sales_order_detail = dlt.readStream("training.piyush.salesorderdetail_silver")
     sales_order_header = dlt.read("training.piyush.salesorderheader_silver")
   
     sales_df = sales_order_detail.join(
         sales_order_header,
         sales_order_detail["SalesOrderID"] == sales_order_header["SalesOrderID"],
         "left"
     )
     sales_df = sales_df.select(
                   sales_order_detail["SalesOrderID"].alias("SalesOrderID"),
                   sales_order_detail["SalesOrderDetailID"].alias("SalesOrderDetailID"),
                   sales_order_detail["ProductID"],
                   sales_order_detail["OrderQty"].alias("OrderQty"),
                   sales_order_detail["UnitPrice"].alias("UnitPrice"),
                   sales_order_detail["UnitPriceDiscount"].alias("UnitPriceDiscount"),
                   sales_order_detail["LineTotal"].alias("LineTotal"),
                   sales_order_header["CustomerID"],
                   sales_order_header["OrderDate"].alias("OrderDate"),
                   sales_order_header["ShipDate"].alias("ShipDate"),
                   sales_order_header["ShipMethodID"].alias("ShipMethodID")
     )
 
     
     customer_dim_df = dlt.read("training.piyush.customer_dimension_gold").filter(col('__END_AT').isNull())
     product_dim_df = dlt.read("training.piyush.product_dimension_gold").filter(col('__END_AT').isNull())
     date_dim_df = dlt.read("training.piyush.date_dimension")
 
     merged_sales_df = (
         sales_df
         .join(product_dim_df, sales_df["ProductID"] == product_dim_df["ProductID"], "left")
         .join(customer_dim_df, sales_df["CustomerID"] == customer_dim_df["CustomerID"], "left")
         .join(date_dim_df, sales_df["OrderDate"] == date_dim_df["ModifiedDate"], "left")
     )
 
     return merged_sales_df.select(
         sales_df["SalesOrderID"].alias("order_id"),
         sales_df["SalesOrderDetailID"].alias("order_detail_id"),
         product_dim_df["Product_Surrogate"].alias("product_surrogate_key"),
         customer_dim_df["Customer_Surrogate"].alias("customer_surrogate_key"),
         sales_df["ProductID"].alias("product_key"),    # FK to dim_product
         sales_df["CustomerID"].alias("customer_key"),  # FK to dim_customer
         sales_df["OrderDate"].alias("date_key"),       # FK to dim_date
         sales_df["OrderQty"],
         sales_df["UnitPrice"],
         sales_df["UnitPriceDiscount"],
         sales_df["LineTotal"],
         customer_dim_df["Person_ModifiedDate"].alias("Customer_ModifiedDate"),
         product_dim_df["Product_ModifiedDate"].alias("Product_ModifiedDate")
      )

**KPI Sales per year per Month**

In [0]:
# @dlt.table(
#     name="kpi1_sales_per_yr_per_month",
#     comment="Sales per year per Month"
# )
# def sales_per_yr_per_month():
#     salesdf = dlt.read("training.piyush.Sales_fact_table")
#     salesdfv1 = salesdf.withColumn('Year', year(to_date(salesdf['date_key'], 'yyyy-MM-DD')))\
#                          .withColumn('Month', month(to_date(salesdf['date_key'], 'yyyy-MM-DD')))
#     return salesdfv1.groupBy('Year', 'Month').agg(count('SalesOrderID').alias('orders_per_month')).orderBy(col('orders_per_month').desc())

**Number of Orders per customer**

In [0]:
# @dlt.table(
#     name="Number_of_Orders_per_customer",
#     comment="Number of Orders per customer"
# )
# def Number_of_Orders_per_customer():
#    salesdf = dlt.read("training.piyush.Sales_fact_table")
      
#    return salesdf.groupBy('customer_key').agg(count('SalesOrderID').alias('orders_per_customer'))


**KPI Avg_unitprice_per_Product**

In [0]:
@dlt.table(
    name="Avg_Unit_Price_per_product",
    comment="Avg Unit Price per product"
)
def Avg_Unit_Price_per_product():
   salesdf = dlt.read("training.piyush.Sales_fact_table")
      
   return salesdf.groupBy('product_key').agg(avg('UnitPrice').alias('avg_UnitPrice'))
   


**Average Value spent per customer for all his products listed**

In [0]:
@dlt.table(
    name="Avgvalue_spent",
    comment="Average Value spent per customer for all his products listed"
)
def Avgvalue_spented():
   salesdf = dlt.read("training.piyush.Sales_fact_table")
      
   return salesdf.groupBy('customer_key').agg(avg('UnitPrice').alias('Avgvalue_spent'))

